<a href="https://colab.research.google.com/github/2303A51317/AIML_2303A51317/blob/main/Lab_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import namedtuple,Counter,defaultdict
import random
import math
import functools
cache=functools.lru_cache(10**6)

 Implement the AI Game Strategy
 Part 1 (a). Install the Python Libraries required for Game Strategy. [CO2]
 1. Install the python libraries- collections, random, math, functools,
 cache = functools.lru cache(10**6)
 2. Implement a Game Class Constructor using action, is terminal, result, utility functions
 3. A game is similar to a problem, but it has a terminal test instead of a goal test, and a
 utility for each terminal state.
 4. Create a game subclass and implement actions, result, is terminal, and utility.
 5. You will also need to set the initial attribute to the initial state; this can be done in the
 constructor

In [8]:
 class Game:
  def actions(self, state):
    ##Return a collection of the allowable moves from this state.
    raise NotImplementedError
  def result(self, state, move):
    ##Return the state that results from making a move from a state.
    raise NotImplementedError
  def is_terminal(self, state):
    ##Return True if this is a final state for the game.
    return not self.actions(state)
  def utility(self, state, player):
    ##Return the value of this final state to player.
    raise NotImplementedError

 Implement a Player Game using the Game Class Constructor.

In [17]:
def play_game(game, strategies: dict, verbose=False):
  ##Play a turn−taking game. strategies is a player name: function dict,
  ##where function(state, game) is used to get the player s move.
  state = game.initial
  while not game.is_terminal(state):
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state, move)
    if verbose:
      print( Player , player, 'move:' , move)
      print(state)
      return state

 Part 2 Implement the Game Strategy Algorithms. [CO3]
 1. Implement the MiniMax Search Algorithm

In [39]:
 def  minimax_search(game, state):
  ##Search game tree to determine best move; return (value, move) pair.
    player = state.to_move
    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, best_move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, best_move = v2, a
        return v, best_move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, best_move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, best_move = v2, a
        return v, best_move

    player = state.to_move
    return max_value(state)

2. Implement the Alpha-Beta Search Algorithm

In [48]:
 def alphabeta_search(game, state):
  ##Search game to determine best action; use alpha−beta pruning.
  ##Search all the way to the leaves.
    player = state.to_move
 def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
            v, best_move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, best_move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, best_move
        return v, best_move

 def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, best_move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, best_move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, best_move
        return v, best_move
        player = state.to_move
        return max_value(state, -math.inf, math.inf)

 Part 3 Implement the Game Strategy using TicTocToe. [CO4]
 1. Implement TicToCToe game using
 display constructors
 init , actions, result, is terminal, utility

In [49]:
  class Board:
    def __init__(self, height, width, to_move, utility):
        self.height = height
        self.width = width
        self.to_move = to_move
        self.utility = utility
        self.squares = {(x, y): None for x in range(width) for y in range(height)}

    def new(self, square, to_move):
        new_board = Board(self.height, self.width, to_move, self.utility)
        new_board.squares = self.squares.copy()
        new_board.squares[square] = to_move
        return new_board

    def __contains__(self, square):
        return self.squares.get(square) is None

    def __len__(self):
        return sum(1 for value in self.squares.values() if value is None)

    def __str__(self):
        return '\n'.join(
            ' '.join(self.squares.get((x, y), '.') for x in range(self.width))
            for y in range(self.height)
        )

class TicTacToe(Game):
    def __init__(self, height=3, width=3, k=3):
        self.k = k  # Number of pieces in a row needed to win
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        # Legal moves are any squares that are not yet taken
        return self.squares - set(board.squares.keys())

    def result(self, board, square):
        # Place a marker for the current player on the square
        player = board.to_move
        new_board = board.new(square, 'O' if player == 'X' else 'X')

        # Check for a win
        win = self.k_in_row(new_board, player, square)
        new_board.utility = (0 if not win else (1 if player == 'X' else -1))

        return new_board

    def utility(self, board, player):
        # Return the utility value to the player
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        # A board is terminal if it is won or there are no empty squares
        return board.utility != 0 or len(board) == 0

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square):
        def in_row(x, y, dx, dy):
            count = 0
            while 0 <= x < board.width and 0 <= y < board.height and board.squares.get((x, y)) == player:
                count += 1
                x += dx
                y += dy
            return count

        x, y = square
        return any(
            in_row(x, y, dx, dy) + in_row(x, y, -dx, -dy) - 1 >= self.k
            for dx, dy in ((0, 1), (1, 0), (1, 1), (1, -1))
        )

 Implement a Game Board using defaultdict using init , new, missing , hash ,
 repr

In [53]:
 class Board(defaultdict):
  empty = '.'
  off = '#'
  def _init_ (self , width=8, height=8, to_move =None , **kwds):
    self._dict_. update (width=width , height=height , to_move = to_move , **kwds)
  def new(self, changes: dict, **kwds)->'Board' :
     board = Board(width=self.width, height=self.height, **kwds)
     board.update(self)
     board.update(changes)
     return board
  def missing (self, loc):
    x, y = loc
    if 0 <= x < self.width and 0 <= y < self.height:
      return self.empty
    else:
      return self.off
  def hash (self):
     return hash(tuple(sorted(self.items()))) + hash(self.to_move)
  def repr(self):
    def row(y):
       return ' '.join(self[x, y] for x in range(self.width))
    return '\n' .join(map(row, range(self.height))) + '\n'

 Implement random player(game,state) and player(search algorithm)

In [57]:
 def random_player(game, state):
  return random.choice(list(game.actions(state)))
 def player(search_algorithm):
  return lambda game, state: search_algorithm(game, state)[1]